In [61]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [62]:
product_type = "chilled"
num_time_periods = 365

In [63]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
article_data = article_data.head(465)

In [64]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 20
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 50
max_order_exceed_multiplier = 1.5

In [65]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [66]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [67]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [68]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [69]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

#### Heuristic

In [70]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [71]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = 0
        self.avg_cost = 0
        self.extra_orders = {}
        self.clearance_cost = 0

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

In [72]:
n = len(demand.keys())
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    clearance_cost = 0
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            if(cell.order_date + shelf_life[item] < t):
                # item cannot be ordered on order_date because it would go bad by t
                if(t not in cell.extra_orders.keys()):
                    cell.extra_orders[t] = {}
                cell.extra_orders[t][item] = math.ceil(demand[t][item] / cu_per_tu[item])
                clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]
            else:   
                cell.orders[item] += demand[t][item]
                # cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    for item in cell.orders.keys():
        cell.orders[item] = math.ceil(cell.orders[item]/cu_per_tu[item])
    cell.clearance_cost = clearance_cost

def fetchExtraVolume(cell, t):
    extra_volume = 0
    if(t in cell.extra_orders.keys()):
        for item in cell.extra_orders[t].keys():
            extra_volume += cell.extra_orders[t][item] * cu_per_tu[item] * volume_per_cu[item] 
    return extra_volume
            
def getCost(cell):
    buff_cost = 0
    getOrders(cell)
    cost = cell.clearance_cost
    volume = 0
    for item in cell.orders.keys():
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        if(cell.orders[item] > maximum_order_quantity_tu[item]):
            cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])
        for extra_order_date in cell.extra_orders.keys():
            for extra_item in cell.extra_orders[extra_order_date].keys():
                cost += (ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]) + ordering_cost_fixed[extra_item]
                if(cell.extra_orders[extra_order_date][extra_item] > maximum_order_quantity_tu[extra_item]):
                    cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    
        volume += cell.orders[item] * cu_per_tu[item] * volume_per_cu[item]
    flag = False
    for t in range(cell.order_date,cell.period_end+1):
        if(volume + fetchExtraVolume(cell, t) > warehouse_volume):
            # CHECK
            cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
            # CHECK
            volume += fetchExtraVolume(cell, t)
        else:
            break

    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1)
    return buff_cost

total_buffer_cost = 0
order_date = 0
end_reached = False
obj_val = 0
schedule = {}
while(True):
    if(order_date%7==6):
        order_date += 1
        continue
    min_cost = float('inf')
    for t in range(order_date,n):
        if(t%7==6):
            continue
        schedule[order_date] = {}
        total_buffer_cost = getCost(cost_matrix[order_date][t])
        if(cost_matrix[order_date][t].avg_cost < min_cost):
            min_cost = cost_matrix[order_date][t].avg_cost
            if(t==n-1):
                schedule[order_date] = cost_matrix[order_date][t].orders
                end_reached = True
                obj_val += cost_matrix[order_date][t].cost
                break
        else:
            schedule[order_date] = cost_matrix[order_date][t-1].orders
            obj_val += cost_matrix[order_date][t-1].cost
            order_date = t
            break
    if(order_date >= n or end_reached):
        break
# print(schedule)
obj_val

907201.8999999999